In [2]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import secrets

In [3]:
def new_key(byteLength=16):
    key = get_random_bytes(byteLength)
    return key

In [4]:
def new_flag(byte_length=16):
    with open('/usr/share/dict/words') as w:
		
        numWords = 8
        words = [word.strip() for word in w]

        flag = ' '.join(secrets.choice(words) for i in range(numWords))
    return flag[0:byte_length]

In [5]:
k = new_key()
secret = new_flag()
cipher = AES.new(k, AES.MODE_ECB)
padding_style = 'pkcs7' # iso7816 # x923
block_size = 16

In [31]:
def decrypt(ciphertext):
    c = unpad(ciphertext, padding_style)
    return cipher.decrypt(c)

def oracle(chosen_plaintext):
    plaintext = chosen_plaintext + secret
    plaintext_bytes = bytes(plaintext, 'utf-8')
    plaintext_padded_bytes = pad(plaintext_bytes, block_size, padding_style)
    ciphertext = cipher.encrypt(plaintext_padded_bytes)
    return ciphertext

First, let's try guessing just one byte of the secret.

In [83]:
# we can check block_size by changing the length of the stencil
stencil = 'A'*15
oracle_reference = oracle(stencil)
print(f'response length (bytes): {len(oracle_reference.hex())/2}')

response length (bytes): 32.0


In [91]:
# we only want the first byte
stencil = 'A'*15
oracle_reference = oracle(stencil)[0:block_size]

secret_revealed = ''

for i in range(256):

    guess = stencil+chr(i)
    oracle_guess = oracle(guess)[0:block_size]
    
    if oracle_guess == oracle_reference:
        print(i)
        secret_revealed += chr(i)
        break

print(f'secret revealed: {secret_revealed}')

 
b'\x00'
b'AB\x00'
102
secret revealed: f


In [89]:
def reveal_one_byte(stencil, secret_revealed, oracle_reference):
    
    for i in range(256):
        guess = stencil+secret_revealed+chr(i)
        #print(f'stencil: {stencil}')
        #print(f'guess: {guess}')
        oracle_guess = oracle(guess)[0:block_size]

        if oracle_guess == oracle_reference:
            secret_byte_revealed = chr(i)
            break
        
    return(secret_byte_revealed, i)

In [90]:
secret_revealed = ''
number_of_tries = []

for j in range(block_size):
    
    stencil = 'A'*(15-j)
    
    oracle_reference = oracle(stencil)[0:block_size]
    secret_byte, effort = reveal_one_byte(stencil, secret_revealed, oracle_reference)
    number_of_tries.append(effort)
    secret_revealed += secret_byte
    
    
print(f'''secret revealed: {secret_revealed}''')
#print(cipher.decrypt(o))


secret revealed: freckling allspi


In [85]:
# finally, double-check our answer is correct:
print(secret)

freckling allspi
